In [1]:
import cv2
import torch
import imutils
import time
import numpy as np
import sys
from face_detection.face_alignment import FaceAlignment

import glob
import os
from os.path import join, dirname, abspath
from models.FaceNet import FaceNet
from registration_database.RegistrationDatabase import RegistrationDatabase

In [4]:
read_root = os.path.join(".\IBM_face_embedding\data\lfw\lfw")
write_root = os.path.join('.\IBM_face_embedding\data\lfw_self_aligned_cropped')
#root2 = os.path.join('.', 'data', 'lfw_crop')

#fa = FaceAlignment()
#cropped_aligned_inference = fa.align(cropped_inference)
#img_item = "aligned-image.png"
#cv2.imwrite(img_item, cropped_aligned_inference)

#people = []

for people in glob.glob(read_root + "/*"):
    name = people.split('\\')[-1]
    write_folder = os.path.join(write_root, name)
    if not os.path.exists(write_folder):
        os.mkdir(write_folder)
    for picture in glob.glob(people + "/*"):
        #picture_img = cv2.imread(picture)
        
        fd = FaceDetect()
        fd.detect()
        fd.crop()
        #fa = FaceAlignment()
        
        #cropped_aligned_img = fa.align(cropped_inference)
        
        #write_filename = "{picture}.png".format(picture=picture)
        #write_filename_root = os.path.join(write_folder, write_filename)
        #cv2.imwrite(write_filename_root, picture_img)
        print(name)
        print(picture)
        print(people)
        
people = []
glob.glob(root + "/*")



Aaron_Eckhart
.\IBM_face_embedding\data\lfw\lfw\Aaron_Eckhart\Aaron_Eckhart_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Aaron_Eckhart
Aaron_Guiel
.\IBM_face_embedding\data\lfw\lfw\Aaron_Guiel\Aaron_Guiel_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Aaron_Guiel
Aaron_Patterson
.\IBM_face_embedding\data\lfw\lfw\Aaron_Patterson\Aaron_Patterson_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Aaron_Patterson
Aaron_Peirsol
.\IBM_face_embedding\data\lfw\lfw\Aaron_Peirsol\Aaron_Peirsol_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Aaron_Peirsol
Aaron_Peirsol
.\IBM_face_embedding\data\lfw\lfw\Aaron_Peirsol\Aaron_Peirsol_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Aaron_Peirsol
Aaron_Peirsol
.\IBM_face_embedding\data\lfw\lfw\Aaron_Peirsol\Aaron_Peirsol_0003.jpg
.\IBM_face_embedding\data\lfw\lfw\Aaron_Peirsol
Aaron_Peirsol
.\IBM_face_embedding\data\lfw\lfw\Aaron_Peirsol\Aaron_Peirsol_0004.jpg
.\IBM_face_embedding\data\lfw\lfw\Aaron_Peirsol
Aaron_Pena
.\IBM_face_embedding\data\lfw\lfw\Aaron_Pena\Aaron_Pena_00

Andy_Wisecarver
.\IBM_face_embedding\data\lfw\lfw\Andy_Wisecarver\Andy_Wisecarver_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Andy_Wisecarver
Anette_Hosoi
.\IBM_face_embedding\data\lfw\lfw\Anette_Hosoi\Anette_Hosoi_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Anette_Hosoi
AnFernce_Negron
.\IBM_face_embedding\data\lfw\lfw\AnFernce_Negron\AnFernce_Negron_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\AnFernce_Negron
Angela_Alvarado_Rosa
.\IBM_face_embedding\data\lfw\lfw\Angela_Alvarado_Rosa\Angela_Alvarado_Rosa_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Angela_Alvarado_Rosa
Angela_Bassett
.\IBM_face_embedding\data\lfw\lfw\Angela_Bassett\Angela_Bassett_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Angela_Bassett
Angela_Bassett
.\IBM_face_embedding\data\lfw\lfw\Angela_Bassett\Angela_Bassett_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Angela_Bassett
Angela_Bassett
.\IBM_face_embedding\data\lfw\lfw\Angela_Bassett\Angela_Bassett_0003.jpg
.\IBM_face_embedding\data\lfw\lfw\Angela_Bassett
Angela_Bassett
.\

Bobby_Bowden
.\IBM_face_embedding\data\lfw\lfw\Bobby_Bowden\Bobby_Bowden_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Bobby_Bowden
Bobby_Goldwater
.\IBM_face_embedding\data\lfw\lfw\Bobby_Goldwater\Bobby_Goldwater_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Bobby_Goldwater
Bobby_Goldwater
.\IBM_face_embedding\data\lfw\lfw\Bobby_Goldwater\Bobby_Goldwater_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Bobby_Goldwater
Bobby_Jackson
.\IBM_face_embedding\data\lfw\lfw\Bobby_Jackson\Bobby_Jackson_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Bobby_Jackson
Bobby_Kielty
.\IBM_face_embedding\data\lfw\lfw\Bobby_Kielty\Bobby_Kielty_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Bobby_Kielty
Bobby_Robson
.\IBM_face_embedding\data\lfw\lfw\Bobby_Robson\Bobby_Robson_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Bobby_Robson
Bobby_Robson
.\IBM_face_embedding\data\lfw\lfw\Bobby_Robson\Bobby_Robson_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Bobby_Robson
Bobo_Balde
.\IBM_face_embedding\data\lfw\lfw\Bobo_Balde\Bobo_Balde_00

Cheryl_James
.\IBM_face_embedding\data\lfw\lfw\Cheryl_James\Cheryl_James_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Cheryl_James
Cheryl_Little
.\IBM_face_embedding\data\lfw\lfw\Cheryl_Little\Cheryl_Little_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Cheryl_Little
Cheryl_Tiegs
.\IBM_face_embedding\data\lfw\lfw\Cheryl_Tiegs\Cheryl_Tiegs_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Cheryl_Tiegs
Chhouk_Rin
.\IBM_face_embedding\data\lfw\lfw\Chhouk_Rin\Chhouk_Rin_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Chhouk_Rin
Chick_Hearn
.\IBM_face_embedding\data\lfw\lfw\Chick_Hearn\Chick_Hearn_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Chick_Hearn
Chick_Hearn
.\IBM_face_embedding\data\lfw\lfw\Chick_Hearn\Chick_Hearn_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Chick_Hearn
Chick_Hearn
.\IBM_face_embedding\data\lfw\lfw\Chick_Hearn\Chick_Hearn_0003.jpg
.\IBM_face_embedding\data\lfw\lfw\Chick_Hearn
Chin-Feng_Chen
.\IBM_face_embedding\data\lfw\lfw\Chin-Feng_Chen\Chin-Feng_Chen_0001.jpg
.\IBM_face_embedding\data

Dan_Boyle
.\IBM_face_embedding\data\lfw\lfw\Dan_Boyle\Dan_Boyle_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Dan_Boyle
Dan_Bylsma
.\IBM_face_embedding\data\lfw\lfw\Dan_Bylsma\Dan_Bylsma_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Dan_Bylsma
Dan_Dickau
.\IBM_face_embedding\data\lfw\lfw\Dan_Dickau\Dan_Dickau_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Dan_Dickau
Dan_Duquette
.\IBM_face_embedding\data\lfw\lfw\Dan_Duquette\Dan_Duquette_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Dan_Duquette
Dan_Guerrero
.\IBM_face_embedding\data\lfw\lfw\Dan_Guerrero\Dan_Guerrero_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Dan_Guerrero
Dan_Kellner
.\IBM_face_embedding\data\lfw\lfw\Dan_Kellner\Dan_Kellner_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Dan_Kellner
Dan_LaCoutre
.\IBM_face_embedding\data\lfw\lfw\Dan_LaCoutre\Dan_LaCoutre_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Dan_LaCoutre
Dan_Monson
.\IBM_face_embedding\data\lfw\lfw\Dan_Monson\Dan_Monson_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Dan_Monson
Dan_Mora

...

Sanja_Papic
.\IBM_face_embedding\data\lfw\lfw\Sanja_Papic\Sanja_Papic_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Sanja_Papic
Santiago_Botero
.\IBM_face_embedding\data\lfw\lfw\Santiago_Botero\Santiago_Botero_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Santiago_Botero
San_Lan
.\IBM_face_embedding\data\lfw\lfw\San_Lan\San_Lan_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\San_Lan
Saoud_Al_Faisal
.\IBM_face_embedding\data\lfw\lfw\Saoud_Al_Faisal\Saoud_Al_Faisal_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Saoud_Al_Faisal
Saparmurat_Niyazov
.\IBM_face_embedding\data\lfw\lfw\Saparmurat_Niyazov\Saparmurat_Niyazov_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Saparmurat_Niyazov
Saparmurat_Niyazov
.\IBM_face_embedding\data\lfw\lfw\Saparmurat_Niyazov\Saparmurat_Niyazov_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Saparmurat_Niyazov
Sarah_Canale
.\IBM_face_embedding\data\lfw\lfw\Sarah_Canale\Sarah_Canale_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Sarah_Canale
Sarah_Hughes
.\IBM_face_embedding\data\lfw\lfw\Sa

Steve_Largent
.\IBM_face_embedding\data\lfw\lfw\Steve_Largent\Steve_Largent_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Steve_Largent
Steve_Lavin
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin\Steve_Lavin_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin
Steve_Lavin
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin\Steve_Lavin_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin
Steve_Lavin
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin\Steve_Lavin_0003.jpg
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin
Steve_Lavin
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin\Steve_Lavin_0004.jpg
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin
Steve_Lavin
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin\Steve_Lavin_0005.jpg
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin
Steve_Lavin
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin\Steve_Lavin_0006.jpg
.\IBM_face_embedding\data\lfw\lfw\Steve_Lavin
Steve_Lenard
.\IBM_face_embedding\data\lfw\lfw\Steve_Lenard\Steve_Lenard_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\S

Tony_Bennett
.\IBM_face_embedding\data\lfw\lfw\Tony_Bennett\Tony_Bennett_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Tony_Bennett
Tony_Bennett
.\IBM_face_embedding\data\lfw\lfw\Tony_Bennett\Tony_Bennett_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Tony_Bennett
Tony_Bennett
.\IBM_face_embedding\data\lfw\lfw\Tony_Bennett\Tony_Bennett_0003.jpg
.\IBM_face_embedding\data\lfw\lfw\Tony_Bennett
Tony_Bennett
.\IBM_face_embedding\data\lfw\lfw\Tony_Bennett\Tony_Bennett_0004.jpg
.\IBM_face_embedding\data\lfw\lfw\Tony_Bennett
Tony_Blair
.\IBM_face_embedding\data\lfw\lfw\Tony_Blair\Tony_Blair_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Tony_Blair
Tony_Blair
.\IBM_face_embedding\data\lfw\lfw\Tony_Blair\Tony_Blair_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Tony_Blair
Tony_Blair
.\IBM_face_embedding\data\lfw\lfw\Tony_Blair\Tony_Blair_0003.jpg
.\IBM_face_embedding\data\lfw\lfw\Tony_Blair
Tony_Blair
.\IBM_face_embedding\data\lfw\lfw\Tony_Blair\Tony_Blair_0004.jpg
.\IBM_face_embedding\data\lfw\lfw\Tony_Blair


Zelma_Novelo
.\IBM_face_embedding\data\lfw\lfw\Zelma_Novelo\Zelma_Novelo_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Zelma_Novelo
Zeng_Qinghong
.\IBM_face_embedding\data\lfw\lfw\Zeng_Qinghong\Zeng_Qinghong_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Zeng_Qinghong
Zhang_Wenkang
.\IBM_face_embedding\data\lfw\lfw\Zhang_Wenkang\Zhang_Wenkang_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Zhang_Wenkang
Zhang_Wenkang
.\IBM_face_embedding\data\lfw\lfw\Zhang_Wenkang\Zhang_Wenkang_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Zhang_Wenkang
Zhang_Yimou
.\IBM_face_embedding\data\lfw\lfw\Zhang_Yimou\Zhang_Yimou_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Zhang_Yimou
Zhang_Ziyi
.\IBM_face_embedding\data\lfw\lfw\Zhang_Ziyi\Zhang_Ziyi_0001.jpg
.\IBM_face_embedding\data\lfw\lfw\Zhang_Ziyi
Zhang_Ziyi
.\IBM_face_embedding\data\lfw\lfw\Zhang_Ziyi\Zhang_Ziyi_0002.jpg
.\IBM_face_embedding\data\lfw\lfw\Zhang_Ziyi
Zhang_Ziyi
.\IBM_face_embedding\data\lfw\lfw\Zhang_Ziyi\Zhang_Ziyi_0003.jpg
.\IBM_face_embedding\data\lfw

['.\\IBM_face_embedding\\data\\lfw\\lfw\\Aaron_Eckhart',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Aaron_Guiel',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Aaron_Patterson',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Aaron_Peirsol',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Aaron_Pena',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Aaron_Sorkin',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Aaron_Tippin',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abbas_Kiarostami',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abba_Eban',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abdel_Aziz_Al-Hakim',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abdel_Madi_Shabneh',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abdel_Nasser_Assidi',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abdoulaye_Wade',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abdulaziz_Kamilov',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abdullah',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abdullah_Ahmad_Badawi',
 '.\\IBM_face_embedding\\data\\lfw\\lfw\\Abdullah_al-Attiyah',
 '.\\I

In [11]:
root = os.path.join(".\IBM_face_embedding\data\lfw\lfw")

root2 = os.path.join('.', 'data', 'lfw_crop')
print(root)
print(root2)

.\IBM_face_embedding\data\lfw\lfw\Aaron_Eckhart\Aaron_Eckhart_0001.jpg
.\data\lfw_crop


In [17]:
root = os.path.join(".\IBM_face_embedding\data\lfw\lfw\Aaron_Eckhart\Aaron_Eckhart_0001.jpg")
img = cv2.imread(root)
img_item = "aligned-image.png"
cv2.imwrite(img_item, img)

#img.shape
#print(img)

True

In [18]:
picture = 234
print("{picture}.png".format(picture=picture))

234.png
